## Reference Links

https://github.com/microsoft/semantic-link-labs

https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.admin.html

https://github.com/microsoft/semantic-link-labs/blob/main/notebooks/Service%20Principal.ipynb


## Import Libraries

In [1]:
import sempy
import sempy.fabric as fabric
import sempy_labs as labs
from pyspark.sql.functions import col, count
from datetime import datetime, timedelta

StatementMeta(, 8d7d0347-274e-4082-865a-5871c56a944e, 5, Finished, Available, Finished)

In [3]:
spark.conf.set("spark.databricks.delta.properties.defaults.minWriterVersion", 5)
spark.conf.set("spark.databricks.delta.properties.defaults.minReaderVersion", 2)

StatementMeta(, 24bf41f2-aedc-4fdd-a209-414b2332384a, 7, Finished, Available, Finished)

#### Get workspaces

In [5]:
df_workspaces = labs.admin.list_workspaces()

spark_df_workspaces = spark.createDataFrame(df_workspaces)

spark_df_workspaces.write.format("Delta").mode("Overwrite").option("delta.columnMapping.mode", "name").saveAsTable("dim_workspaces")
# spark_df_workspaces.write.format("Delta").mode("Overwrite").save("abfss-path")


StatementMeta(, e62faebd-125e-47af-9801-364e18d443c9, 9, Finished, Available, Finished)

#### Get reports

In [4]:
df_reports = labs.admin.list_reports()

# Create a spark dataframe
spark_df_report = spark.createDataFrame(df_reports)

# Overwrite data into the delta table
spark_df_report.write.format("Delta").mode("Overwrite").option("delta.columnMapping.mode", "name").saveAsTable("dim_reports")
# spark_df_report.write.format("Delta").mode("Overwrite").save("abfss-path")

StatementMeta(, fe33ed02-6665-409b-ac26-975472e3eaaa, 8, Finished, Available, Finished)

#### Get datasets

In [2]:
df_datasets = labs.admin.list_datasets()
# display(df_datasets)

# Create a spark dataframe
spark_df_datasets = spark.createDataFrame(df_datasets)

# Overwrite data into the delta table
spark_df_datasets.write.format("Delta").mode("Overwrite").option("delta.columnMapping.mode", "name").saveAsTable("dim_datasets")
# spark_df_datasets.write.format("Delta").mode("Overwrite").save("abfss-path")

StatementMeta(, 8d7d0347-274e-4082-865a-5871c56a944e, 6, Finished, Available, Finished)

#### Get Fabric Items

In [3]:
df_lists = labs.admin.list_items()
# display(df_lists)

# Create a spark dataframe
spark_df_lists = spark.createDataFrame(df_lists)

# Overwrite data into the delta table
spark_df_lists.write.format("Delta").mode("Overwrite").option("delta.columnMapping.mode", "name").saveAsTable("dim_items")
# spark_df_datasets.write.format("Delta").mode("Overwrite").save("abfss-path")

StatementMeta(, 8d7d0347-274e-4082-865a-5871c56a944e, 7, Finished, Available, Finished)

#### Get Domains

In [ ]:
df_domains = labs.admin.list_domains()
# display(df_datasets)

# Create a spark dataframe
spark_df_domains = spark.createDataFrame(df_domains)

# Overwrite data into the delta table
spark_df_domains.write.format("Delta").mode("Overwrite").option("delta.columnMapping.mode", "name").saveAsTable("dim_domains")
# spark_df_datasets.write.format("Delta").mode("Overwrite").save("abfss-path")